In [11]:
"""
Simple Carbon Reduction Optimizer (Always-On, Slack Variables, City-Specific Targets)
- Always optimizes (even when PM2.5 < 15) by pulling toward WHO 5 µg/m³
- Uses shortfall slack v >= 0 for feasibility
- Light diversification so results don't collapse
- Deterministic city-based cost tie-breaker to avoid identical blends
"""

import requests
import numpy as np
from scipy.optimize import linprog
from datetime import datetime, timezone

# -----------------------------
# CONFIG
# -----------------------------
PM_TARGET    = 15.0     # policy target (µg/m³)
WHO_GUIDE    = 5.0      # WHO guideline (µg/m³)
PULL_FRACTION = 0.4     # when below 15, aim to close 40% of the gap to 5
SCALE        = 1.0     # µg/m³ → "reduction units" scaling

# Effects (reduction units per unit x) and costs ($ per unit x)
EFFECTS = np.array([2.5, 4.0, 1.5])      # transport, industry, residential
BASE_COSTS = np.array([150.0, 200.0, 100.0])

# Diversification: any single sector ≤ P_MAX of achieved reduction
P_MAX = 0.7

# Bounds
BOUNDS = [(0, 40), (0, 30), (0, 50)]     # x1, x2, x3

# Slack penalty
SLACK_PENALTY = 1e4

# Optional OpenAQ key
OPENAQ_API_KEY = None

# -----------------------------
# API HELPERS
# -----------------------------
def _openaq_v3_headers():
    h = {"Accept": "application/json"}
    if OPENAQ_API_KEY:
        h["X-API-Key"] = OPENAQ_API_KEY
    return h

def fetch_pm25_from_openaq_v3(city):
    base = "https://api.openaq.org/v3"
    s = requests.Session(); s.headers.update(_openaq_v3_headers())
    r = s.get(f"{base}/locations", params={"city": city, "limit": 50, "sort": "desc"}, timeout=20)
    r.raise_for_status()
    for loc in r.json().get("results", []):
        loc_id = loc.get("id") or loc.get("locationId") or loc.get("locationsId")
        if not loc_id: continue
        lr = s.get(f"{base}/locations/{loc_id}/latest", params={"limit": 100}, timeout=20)
        if lr.status_code != 200: continue
        for meas in lr.json().get("results", []):
            p = (meas.get("parameter") or {}).get("name","").lower()
            if p in {"pm25","pm2.5","pm_2_5","pm2_5"}:
                return {
                    "city": city,
                    "pm25": float(meas.get("value")),
                    "unit": (meas.get("parameter") or {}).get("units","µg/m³"),
                    "when_utc": (meas.get("date") or {}).get("utc") or (meas.get("date") or {}).get("local"),
                    "source": "OpenAQ v3",
                    "location_name": loc.get("name") or loc.get("city") or str(loc_id)
                }
    return None

def geocode_city_openmeteo(city):
    r = requests.get("https://geocoding-api.open-meteo.com/v1/search",
                     params={"name": city, "count": 1, "language": "en", "format": "json"}, timeout=20)
    r.raise_for_status()
    items = (r.json().get("results") or [])
    if not items: return None
    it = items[0]
    return {"lat": it["latitude"], "lon": it["longitude"], "name": it.get("name", city)}

def fetch_pm25_from_openmeteo(city):
    g = geocode_city_openmeteo(city)
    if not g: return None
    r = requests.get("https://air-quality-api.open-meteo.com/v1/air-quality",
                     params={"latitude": g["lat"], "longitude": g["lon"], "hourly": "pm2_5", "timezone": "UTC"}, timeout=20)
    r.raise_for_status()
    js = r.json(); hourly = js.get("hourly") or {}
    times = hourly.get("time") or []; vals = hourly.get("pm2_5") or []
    if not times or not vals: return None
    return {"city": city, "pm25": float(vals[-1]), "unit": "µg/m³", "when_utc": times[-1],
            "source": "Open-Meteo", "location_name": g["name"]}

def fetch_air_quality(city):
    try:
        d = fetch_pm25_from_openaq_v3(city)
        if d: return d
    except Exception:
        pass
    try:
        d = fetch_pm25_from_openmeteo(city)
        if d: return d
    except Exception:
        pass
    return None

# -----------------------------
# CITY-SPECIFIC TARGET & COST NUDGE
# -----------------------------
def city_required_units(pm):
    """
    Above 15: reduce exceedance toward 15.
    Below 15: pull a fraction of the gap toward WHO 5.
    """
    if pm >= PM_TARGET:
        gap = pm - PM_TARGET                                # µg/m³ above 15
        return max(1.0, gap * SCALE)                        # at least a bit of action
    else:
        gap_to_who = max(0.0, pm - WHO_GUIDE)               # how far above 5
        return max(1.0, PULL_FRACTION * gap_to_who * SCALE) # fraction toward 5

def city_tiebreak_costs(city):
    """
    Deterministic, tiny nudge so cities with similar targets
    don't all pick the exact same blend. (No randomness.)
    """
    h = sum(ord(c) for c in city.lower())
    bumps = np.array([
        ((h % 7) + 1) * 1e-3,   # 0.001 .. 0.007
        ((h % 11)+ 1) * 1e-3,   # 0.001 .. 0.011
        ((h % 13)+ 1) * 1e-3    # 0.001 .. 0.013
    ])
    return BASE_COSTS * (1.0 + bumps)

# -----------------------------
# OPTIMIZATION (slack + diversification)
# -----------------------------
def solve_lp(required_units, costs):
    """
    Variables: x1, x2, x3, v, t
    Minimize:  costs·x + SLACK_PENALTY * v
    s.t.:
      EFFECTS·x + v = required_units
      EFFECTS·x - t = 0
      EFFECTS[i]*x_i <= P_MAX * t,  i in {0,1,2}
      bounds on x; v >= 0; t >= 0
    """
    c = np.array([costs[0], costs[1], costs[2], SLACK_PENALTY, 0.0])

    A_eq = np.array([
        [EFFECTS[0], EFFECTS[1], EFFECTS[2], 1.0,  0.0],  # meet target with slack
        [EFFECTS[0], EFFECTS[1], EFFECTS[2], 0.0, -1.0],  # define t
    ])
    b_eq = np.array([required_units, 0.0])

    A_ub = np.array([
        [EFFECTS[0], 0.0,        0.0,        0.0, -P_MAX],
        [0.0,        EFFECTS[1], 0.0,        0.0, -P_MAX],
        [0.0,        0.0,        EFFECTS[2], 0.0, -P_MAX],
    ])
    b_ub = np.zeros(3)

    bounds = [BOUNDS[0], BOUNDS[1], BOUNDS[2], (0, None), (0, None)]

    return linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,
                   bounds=bounds, method="highs")

# -----------------------------
# RUN
# -----------------------------
def fmt_utc(s):
    if not s: return "n/a"
    try:
        dt = datetime.fromisoformat(s.replace("Z","+00:00"))
        return dt.astimezone(timezone.utc).strftime("%Y-%m-%d %H:%M UTC")
    except Exception:
        return str(s)

def run(city):
    print("\n" + "="*60)
    print("CARBON EMISSION REDUCTION OPTIMIZER (Always-On)")
    print("="*60 + "\n")

    aq = fetch_air_quality(city)
    if not aq:
        print(f"❌ No air quality data for '{city}'"); return

    pm = aq["pm25"]
    print(f"Source: {aq['source']}")
    print(f"City/Location: {aq['city']} — {aq['location_name']}")
    print(f"PM2.5: {pm:.2f} {aq['unit']}  (as of {fmt_utc(aq['when_utc'])})")

    required_units = city_required_units(pm)
    costs = city_tiebreak_costs(aq["city"])

    print(f"\nTarget (reduction units): {required_units:.2f}  "
          f"[toward {'15' if pm>=PM_TARGET else 'WHO 5'}]")
    print(f"Diversification cap P_MAX: {P_MAX:.0%}  |  Costs used: {costs.round(2)}")

    res = solve_lp(required_units, costs)
    if not res.success:
        print(f"❌ Optimization failed: {res.message}"); return

    x1, x2, x3, v, t = res.x
    cost = costs[0]*x1 + costs[1]*x2 + costs[2]*x3

    print("\n✓ OPTIMAL SOLUTION FOUND\n")
    print("Plan (units):")
    print(f"  🚗 Transport (x1):   {x1:6.2f}   eff={EFFECTS[0]:.1f}  cost/unit=${costs[0]:.2f}")
    print(f"  🏭 Industry  (x2):   {x2:6.2f}   eff={EFFECTS[1]:.1f}  cost/unit=${costs[1]:.2f}")
    print(f"  🏠 Residential(x3):   {x3:6.2f}   eff={EFFECTS[2]:.1f}  cost/unit=${costs[2]:.2f}")

    contrib = np.array([EFFECTS[0]*x1, EFFECTS[1]*x2, EFFECTS[2]*x3])
    shares = (contrib / t * 100.0) if t > 1e-9 else np.zeros_like(contrib)

    print("\nResults:")
    print(f"  Achieved reduction (t): {t:6.2f}   Required: {required_units:6.2f}")
    print(f"  Shortfall slack v:      {v:6.2f}   (0 means requirement met exactly)")
    print(f"  Intervention cost:      ${cost:,.2f}")
    print(f"  Sector shares:          Transport {shares[0]:.1f}% | Industry {shares[1]:.1f}% | Residential {shares[2]:.1f}%")

if __name__ == "__main__":
    # pip install requests numpy scipy
    while True:
        try:
            city = input("\nEnter a city (or 'q' to quit): ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nGoodbye!"); break
        if city.lower() in {"q","quit","exit"}:
            print("\nGoodbye!"); break
        if city:
            run(city)
        else:
            print("Please enter a valid city name.")



CARBON EMISSION REDUCTION OPTIMIZER (Always-On)

Source: Open-Meteo
City/Location: London — London
PM2.5: 4.60 µg/m³  (as of 2025-11-16 04:00 UTC)

Target (reduction units): 1.00  [toward WHO 5]
Diversification cap P_MAX: 70%  |  Costs used: [151.05 200.4  100.1 ]

✓ OPTIMAL SOLUTION FOUND

Plan (units):
  🚗 Transport (x1):     0.12   eff=2.5  cost/unit=$151.05
  🏭 Industry  (x2):     0.18   eff=4.0  cost/unit=$200.40
  🏠 Residential(x3):     0.00   eff=1.5  cost/unit=$100.10

Results:
  Achieved reduction (t):   1.00   Required:   1.00
  Shortfall slack v:        0.00   (0 means requirement met exactly)
  Intervention cost:      $53.20
  Sector shares:          Transport 30.0% | Industry 70.0% | Residential 0.0%

CARBON EMISSION REDUCTION OPTIMIZER (Always-On)

Source: Open-Meteo
City/Location: Paris — Paris
PM2.5: 12.50 µg/m³  (as of 2025-11-16 04:00 UTC)

Target (reduction units): 3.00  [toward WHO 5]
Diversification cap P_MAX: 70%  |  Costs used: [150.75 201.   101.1 ]

✓ OPTIMAL 